In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate();

  Activating environment at `~/SSD/Code/Julia/tinympc-julia/cartpole_tinympc/Project.toml`


Precompiling 

project...


  ✓ LaTeXStrings


  ✓ ConcurrentUtilities
  ✓ LoggingExtras
  ✓ ConstructionBase
  ✓ URIs


...

  ✓ GR


  ✓ Unitful


  ✓ UnitfulLatexify


  ✓ Plots


  14 dependencies successfully precompiled in 72 seconds (138 already precompiled)


In [2]:
using LinearAlgebra
using BlockDiagonals
using ForwardDiff
using Plots
using Random;
using Printf

In [152]:
#Cartpole parameters: [x, xd, theta, thetad]
M = 0.5
m = 0.2
l = 0.3
I = 0.1
g = 9.8
a12 = m^2 * g * l^2 / ((M+m)*I+M*m*l^2)
a32 = m * g * l * (M+m) / ((M+m)*I+M*m*l^2)
b1 = (I+m*l^2) / ((M+m)*I+M*m*l^2)
b3 = m * l / ((M+m)*I+M*m*l^2)
Ac = [0 1 0 0; 0 0 a12 0; 0 0 0 1; 0 0 a32 0]
Bc = [0; b1; 0; b3]

freq1 = 100
h = 1 / freq1 #50 Hz

Nx = 4     # number of states (quaternion)
Nu = 1     # number of controls
nx = Nx
nu = Nu
Tfinal = 5.0 # final time
Nt = Int(Tfinal / h) + 1    # number of time steps
t_vec = h * (0:Nt-1)
thist = Array(range(0, h * (Nt - 1), step=h));

In [153]:
function quad_dynamics(x, u)
  return Ac*x + Bc*u
end
function quad_dynamics_rk4(x, u, h)
  #RK4 integration with zero-order hold on u
  f1 = quad_dynamics(x, u)
  f2 = quad_dynamics(x + 0.5 * h * f1, u)
  f3 = quad_dynamics(x + 0.5 * h * f2, u)
  f4 = quad_dynamics(x + h * f3, u)
  xn = x + (h / 6.0) * (f1 + 2 * f2 + 2 * f3 + f4)
  return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [154]:
#Goal state, upright position
xg = [0.; 0; 0; 0]; 
ug = 0.;

In [155]:
#Linearize dynamics about hover
A = ForwardDiff.jacobian(x -> quad_dynamics_rk4(x, ug, h), xg)
B = ForwardDiff.derivative(u -> quad_dynamics_rk4(xg, u, h), ug);
function discrete_dynamics(params, x, u, k)
  A * x + B * u
end

discrete_dynamics (generic function with 1 method)

In [156]:
display(A)
display(B)
@show(A')
@show(B)

4×4 Matrix{Float64}:
 1.0  0.01  2.23301e-5  7.44304e-8
 0.0  1.0   0.00446621  2.23301e-5
 0.0  0.0   1.00026     0.0100009
 0.0  0.0   0.0521058   1.00026

4-element Vector{Float64}:
 7.468368562730335e-5
 0.014936765390161838
 3.79763323185387e-5
 0.007595596218554721

A' = [1.0 0.0 0.0 0.0; 0.01 1.0 0.0 0.0; 2.2330083403300767e-5 0.004466210576510177 1.0002605176397052 0.05210579005928538; 7.443037974683548e-8 2.2330083403300767e-5 0.01000086835443038 1.0002605176397052]
B = [7.468368562730335e-5, 0.014936765390161838, 3.79763323185387e-5, 0.007595596218554721]


4-element Vector{Float64}:
 7.468368562730335e-5
 0.014936765390161838
 3.79763323185387e-5
 0.007595596218554721

In [169]:
# IHLQR terminal cost
P = [zeros(nx, nx) for i = 1:1000]   # cost to go quadratic term
p = [zeros(nx) for i = 1:1000]      # cost to go linear term
d = [zeros(nu) for i = 1:1000-1]    # feedforward control
K = [zeros(nu, nx) for i = 1:1000-1] # feedback gain

# Cost weights
# Saved
# max_dev_x = [0.1; 0.1; 0.1;  0.5; 0.5; 0.05;  0.5; 0.5; 0.5;  0.5; 0.5; 0.1]
# max_dev_u = [0.5; 0.5; 0.5; 0.5]/10
# max_dev_x = [5; 5; 0.001; 0.5; 0.5; 0.05; 0.5; 0.5; 0.5; 0.7; 0.7; 0.2] / 1
# max_dev_u = [0.5; 0.5; 0.5; 0.5] / 1
# Q = diagm(1 ./ (max_dev_x .^ 2))
# R = diagm(1 ./ (max_dev_u .^ 2))

Q = [1.0 0 0 0; 0 0.1 0 0; 0 0 1.0 0; 0 0 0 0.1]*10
R = 1.0
Qf = 1 * Q

# IHLQR terminal cost for orginal problem
P[1000] .= Q
for k = (1000-1):-1:1
    K[k] .= (R + B' * P[k+1] * B) \ (B' * P[k+1] * A)
    P[k] .= Q + A' * P[k+1] * (A - B * K[k])
end
Qf = P[1]
display(K[1])
display(P[1])

1×4 Matrix{Float64}:
 -2.45857  -4.16675  40.2322  17.853

4×4 Matrix{Float64}:
  1553.98   1174.13  -5904.03  -2646.95
  1174.13   1279.31  -6875.44  -3085.2
 -5904.03  -6875.44  42758.0   18973.0
 -2646.95  -3085.2   18973.0    8487.34

In [170]:
# PENALTY - important!! Trade-off between obj and constraint
ρ = 0.0

# data for new admm formulation
R̃ = R 
Q̃ = Q 
Q̃f = Qf 

# Precompute
cache = (
    Kinf=zeros(nu, nx),
    Pinf=zeros(nx, nx),
    Kinf2=zeros(nu, nx),
    Pinf2=zeros(nx, nx),
    Quu_inv=zeros(nu, nu),
    Quu_inv2=zeros(nu, nu),
    AmBKt=zeros(nx, nx),
    AmBKt2=zeros(nx, nx), #(Ã - B̃*K[1])',
    coeff_d2p=zeros(nx, nu),
    coeff_d2p2=zeros(nx, nu),
)

# IHLQR terminal cost for new problem
P = [zeros(nx, nx) for i = 1:1000]   # cost to go quadratic term
p = [zeros(nx) for i = 1:1000]      # cost to go linear term
d = [zeros(nu) for i = 1:1000-1]    # feedforward control
K = [zeros(nu, nx) for i = 1:1000-1] # feedback gain

P[1000] .= Q̃f
# For streched dynamics (unused -> set `adaptive_step = 0`)
for k = (1000-1):-1:1
    K[k] .= (R̃ + B' * P[k+1] * B) \ (B' * P[k+1] * A)
    P[k] .= Q̃ + A' * P[k+1] * (A - B * K[k])
end
cache.Kinf .= K[1]
cache.Pinf .= P[1]
cache.AmBKt .= (A - B * K[1])'
cache.Quu_inv .= (R̃ + B' * cache.Pinf * B) \ I
display(cond(R̃ + B' * cache.Pinf * B))
cache.coeff_d2p .= cache.Kinf' * R̃ - cache.AmBKt * cache.Pinf * B


1.0

cache.Kinf .= K[1] = [-3.038285934765668 -4.969170646522577 45.19435775437705 20.089877758824557]
cache.Pinf .= P[1] = [1635.5177732493137 1279.281604440924 -6538.146901457742 -2932.4242188080893; 1279.2816044409426 1419.2335459442818 -7728.018316795391 -3469.2382213306814; -6538.146901457785 -7728.018316795453 47973.525344154645 21322.768546355954; -2932.424218808135 -3469.238221330693 21322.76854635586 9545.982297269238]


4×1 Matrix{Float64}:
 -7.061018436615996e-14
 -8.615330671091215e-14
  2.2737367544323206e-13
  1.7408297026122455e-13

In [171]:
display(cache.Kinf)
display(cache.Pinf)
display(cache.AmBKt)
display(cache.Quu_inv)
display(cache.coeff_d2p)


1×4 Matrix{Float64}:
 -3.03829  -4.96917  45.1944  20.0899

4×4 Matrix{Float64}:
  1635.52   1279.28  -6538.15  -2932.42
  1279.28   1419.23  -7728.02  -3469.24
 -6538.15  -7728.02  47973.5   21322.8
 -2932.42  -3469.24  21322.8    9545.98

4×4 Matrix{Float64}:
  1.00023      0.0453822  0.000115383   0.0230776
  0.0103711    1.07422    0.000188711   0.0377438
 -0.00335295  -0.670591   0.998544     -0.291172
 -0.00150031  -0.300055   0.00923793    0.847666

1×1 Matrix{Float64}:
 0.09231181421394602

4×1 Matrix{Float64}:
 -7.061018436615996e-14
 -8.615330671091215e-14
  2.2737367544323206e-13
  1.7408297026122455e-13

In [159]:
# display(cache.Kinf)
# # C Language for LQR2 alone
# @printf("FOR C:\n");
# str = "static float K[NU][NXt] = {\n"
# for i = 1:4
#   str = str * "  {"
#   for j = 1:12
#     # if abs(K[i, j]) < 1e-6
#     #   K[i, j] = 0.0
#     # end
#     this_str = @sprintf("%.6f", cache.Kinf[i, j])

#     str = str * this_str * "f"
#     if j < 12
#       str = str * ","
#     end
#   end
#   str = str * "},\n"
# end
# str = str * "};"
# @printf("%s",str)

In [160]:
#Feedback tracking controller
include("../LibTinyMPC.jl")
N = 10
dt = 1 / freq1
Tfinal = 10.0 # final time
Nt = Int(Tfinal / h) + 1    # number of time steps
t_vec = h * (0:Nt-1)


#Create trajectory to follow
z_ref = 0.6
Xref = [[zeros(3); 1; zeros(9)] for i = 1:Nt]
Δx̃ref = [zeros(nx) for i = 1:Nt]
ΔUref = [zeros(nu) for i = 1:Nt-1]

## Hovering
# Xref = [xg for i = 1:Nt]

## Landing
# x_landing = [1; 1.2; 0; 1; zeros(9)]
# Xref = [x_landing for i = 1:Nt]

# Number-eight
# Xref = [[1*cos(t)*sin(t);1*cos(t);z_ref;1;zeros(9)] for t = range(-pi/2, 3*pi/2, length = Nt)]  

# Cyclone
t_end = 4*2*pi
Xref = [[0.5*sin(t);0.5*cos(t);z_ref/(3*t/t_end+1);1;zeros(9)] for t = range(0, t_end, length = Nt)]  

## Triangle
# p0 = [0; 0; z_ref];
# p1 = [3; 4; z_ref];
# p2 = [0; 8; z_ref];
# p3 = [0; -2; z_ref];
# range1 = collect(LinRange(p0, p1, round(Int, (Nt) / 3)))
# range2 = collect(LinRange(p1, p2, round(Int, (Nt) / 3)))
# range3 = collect(LinRange(p2, p3, round(Int, (Nt) / 3) + 1))
# range123 = [range1; range2; range3]
# for k = 1:Nt
#     Xref[k][1:3] .= range123[k]
# end

# Square
# p0 = [0; 0; 0.5]; p1 = [2; 0; 0.5]; p2 = [2; 2; 0.5]; p3 = [0; 2; 0.5]
# range1 = collect(LinRange(p0, p1, round(Int,(Nt)/4)))
# range2 = collect(LinRange(p1, p2, round(Int,(Nt)/4)))
# range3 = collect(LinRange(p2, p3, round(Int,(Nt)/4)))
# range4 = collect(LinRange(p3, p0, round(Int,(Nt)/4)+1))
# range1234 = [range1; range2; range3; range4]
# for k = 1:Nt
#     Xref[k][1:3] .= range1234[k]
# end

# Velocity reference: speeds up the tracking but less smooth
for i = 1:Nt-1
    Xref[i][8:10] = (Xref[i+1][1:3] - Xref[i][1:3]) / dt / 1.0
end
for k = 1:Nt
    x = Xref[k]
    q = x[4:7]
    ϕ = qtorp(L(qg)' * q)
    Δx̃ref[k] .= [x[1:3] - rg; ϕ; x[8:10] - vg; x[11:13] - ωg]
end


u_min = -uhover #-0.5*ones(nu)
u_max = 1 .- uhover #0.5*ones(nu)
# u_min = -50.0*ones(nu)
# u_max = 50.0*ones(nu)
# state is x y v θ
x_min = []
# x_min[3] = 0.85
x_max = []
# x_max[1] = 3.05
# Halfspace
Acu = []
bcu = 0.0
Acx = []
bcx = 0.0

# Obstacle
obs_c = [3.1, 4.0, 1.0]
obs_r = 0.5

# previous MPC iterate
X = [zeros(nx) for i = 1:N]
U = [zeros(nu) for k = 1:N-1]
ZU = [zeros(nu) for i = 1:N-1]
ZU_new = [zeros(nu) for i = 1:N-1]
YU = [zeros(nu) for i = 1:N-1]
ZX = [zeros(nx) for i = 1:N]
ZX_new = [zeros(nx) for i = 1:N]
YX = [zeros(nx) for i = 1:N]

params = (
    nx=nx, nu=nu,
    N=N, Q=Q, R=R, Qf=Qf,
    en_box_input=1, u_min=u_min, u_max=u_max, Acu=Acu, bcu=bcu,
    en_box_state=0, x_min=x_min, x_max=x_max, Acx=Acx, bcx=bcx,
    en_hplane_state=0,
    en_box_soc=0, en_soc_state=1, mu=0.8,
    Xref=Δx̃ref, Uref=ΔUref, dt=dt, cache=cache,
);



P = [zeros(nx, nx) for i = 1:N]   # cost to go quadratic term
p = [zeros(nx) for i = 1:N]      # cost to go linear term
d = [zeros(nu) for i = 1:N-1]    # feedforward control
K = [zeros(nu, nx) for i = 1:N-1] # feedback gain
qq = [zeros(nx) for i = 1:N]
r = [zeros(nu) for i = 1:N-1]

verbose = 0
adaptive_step = 0
iters = zeros(10000)
function controller_mpc(params, x, k)
    q = x[4:7]
    ϕ = qtorp(L(qg)' * q)

    Δx̃ = [x[1:3] - rg; ϕ; x[8:10] - vg; x[11:13] - ωg]
    X[1] .= Δx̃

    U = [zeros(nu) for k = 1:N-1]  # reset because previous sln does not converge
    # ZU = [zeros(nu) for i = 1:N-1]
    # ZU_new = [zeros(nu) for i = 1:N-1]
    YU = [zeros(nu) for i = 1:N-1]
    # ZX = [zeros(nx) for i = 1:N]
    # ZX_new = [zeros(nx) for i = 1:N]
    YX = [zeros(nx) for i = 1:N]

    u, status, iters[k] = solve_admm!(params, qq, r, p, d, X, ZX, ZX_new, YX, U, ZU, ZU_new, YU; ρ=ρ, abs_tol=1e-2, max_iter=500, adaptive_step=adaptive_step)
    # u = -cache.Kinf*(Δx̃ - Δx̃ref[k])
    # display(iters[k])
    # status == 0 && display("Max iteration reached")
    return u + uhover
end

#ASK: Do I need to stretch the reference?
function update_reference(params, k)
    Uref_ = 1 * ΔUref[k:N+k-2]
    Xref_ = 1 * Δx̃ref[k:N+k-1]

    if (adaptive_step > 0)
        for i = 3:N-1
            Uref_[i] .= ΔUref[k+1+(i-2)*step_ratio]
            Xref_[i] .= Δx̃ref[k+1+(i-2)*step_ratio]
        end
        Xref_[N] .= Δx̃ref[k+1+(N-2)*step_ratio]
        # for i = 3:N-1
        #     Xref_[i][7:9] = (Xref_[i+1][1:3] - Xref_[i][1:3])/freq2
        # end
    end

    params_new = (; params..., Uref=Uref_, Xref=Xref_)  # update ref
    return params_new
end

function update_constraint(params, X, k)
    Acx_ = [zeros(3) for i = 1:N]  # only three for position
    bcx_ = [1.0 for i = 1:N]
    for k = 1:N
        xp = X[k][1:3]
        vecXC = obs_c - xp
        d = sqrt(vecXC' * vecXC)
        if (d > (3 * obs_r))
            # unactivated if far from obstacle
            # Acx_[k] = zeros(3)
            # bcx_[k] = 1.0
        else
            vecXI = vecXC * (d - obs_r) / d
            Xi = xp + vecXI
            ∇f = 2 * (obs_c - Xi)
            denom = sqrt(∇f' * ∇f)
            Acx_[k] = ∇f / denom
            bcx_[k] = ∇f' * Xi / denom
        end
    end
    params_new = (; params..., Acx=Acx_, bcx=bcx_)  # update ref
    # println(xp, Acx_, bcx_)
    return params_new
end


UndefVarError: UndefVarError: L not defined

In [161]:
# Simulation
uhist = [zeros(Nu) for i in 1:Nt-1]
xhist = [zeros(Nx) for i in 1:Nt]
Random.seed!(12);
# xhist[1] .= [rg+0*randn(3)/3; L(qg)*rptoq([1; 0; 0]); vg; ωg]  # initial conditions
xhist[1] .= Xref[1]
# xhist[1][1:3] .= [0; 0; 3.0]
# xhist[1][1:3] += rand(3) * 0.2;
# xhist[1][1:2] = [0;0]
backward_pass!(Q̃, R̃, P, params, adaptive_step)
Nsim = Nt - 100 - 1
for k = 1:Nsim
# for k = 1:10
    params = update_reference(params, k)
    # params = update_constraint(params, X, k)
    # display(params1.Xref)
    # display(xhist[k])
    # shift_fill(U)
    uhist[k] = controller_mpc(params, xhist[k], k)
    uhist[k] = clamp.(uhist[k], (uhover+u_min)[1], (uhover+u_max)[1])
    xhist[k+1] = quad_dynamics_rk4(xhist[k], uhist[k], h)
end

DimensionMismatch: DimensionMismatch("array could not be broadcast to match destination")

In [162]:
display(maximum(iters))
for i = 1:Nsim
  # xhist[i][3] > 1.0 + 0.01 && error("infeasibilty")
  # @show ( (norm(xhist[i][1:2]) - params.mu * xhist[i][3]))
  (norm(xhist[i][1:2]) - params.mu * xhist[i][3]) > 0.1 && error("infeasibilty", i)
  
end
maximum(xhist)

UndefVarError: UndefVarError: iters not defined

In [163]:
Xsim_m = mat_from_vec(Xref[1:Nsim])
plot(Xsim_m[2, :], Xsim_m[1, :], label="ref", linestyle=:solid, linewidth=2, title="State History", xlabel="y", ylabel="x")
Xsim_m = mat_from_vec(xhist[1:Nsim])
display(plot!(Xsim_m[2, :], Xsim_m[1, :], label="real", linestyle=:dash, linewidth=2, title="State History", xlabel="y", ylabel="x", aspect_ratio=:equal))
# display(scatter!([obs_c[2]], [obs_c[1]], color="red", label="", markersize=70 * obs_r))


UndefVarError: UndefVarError: Nsim not defined

In [164]:
Xsim_m = mat_from_vec(Xref[1:1:Nsim])
plot(Xsim_m[2, :], Xsim_m[1, :], label="ref",
  linestyle=:solid, linewidth=2,
  title="State History", xlabel="y", ylabel="x")
Xsim_m = mat_from_vec(xhist[1:1:Nsim])
display(plot!(Xsim_m[2, :], Xsim_m[1, :], label="real", linestyle=:dash, linewidth=2,
  title="State History", xlabel="y", ylabel="x", aspect_ratio=:equal))

UndefVarError: UndefVarError: Nsim not defined

In [165]:
Xsim_m = mat_from_vec(xhist[1:Nsim] - 0 * Xref[1:Nsim])
Usim_m = mat_from_vec(uhist[1:Nsim-1])
display(plot(Xsim_m[1:7, :]', label=["x" "y" "z" "qw" "qx" "qy" "qz"],
  linestyle=[:solid :solid :solid :dash :dash :dash :dash], linewidth=[2 2 2 2 2 2 2],
  title="State History", xlabel="time (s)", ylabel="x"))
display(plot(Usim_m', label=["u₁" "u₂" "u₃" "u₄"],
  title="Input History", linewidth=[2 2 2 2 2 2 2],
  xlabel="time (s)", ylabel="u"))
norm(xhist[1:Nsim] - 0 * Xref[1:Nsim])

UndefVarError: UndefVarError: Nsim not defined

In [166]:
Xsim_m = mat_from_vec(xhist[1:Nsim])
Usim_m = mat_from_vec(uhist[1:Nsim-1])
display(plot(Xsim_m[1:7, :]', label=["x" "y" "z" "qw" "qx" "qy" "qz"],
  linestyle=[:solid :solid :solid :dash :dash :dash :dash], linewidth=[2 2 2 2 2 2 2],
  title="State History", xlabel="time (s)", ylabel="x"))
display(plot(Usim_m', label=["u₁" "u₂" "u₃" "u₄"],
  title="Input History", linewidth=[2 2 2 2 2 2 2],
  xlabel="time (s)", ylabel="u"))
norm(xhist[1:Nsim] - 0 * Xref[1:Nsim])

UndefVarError: UndefVarError: Nsim not defined

In [167]:
# function export_mat_to_eigen(prefix, data)
#   str = prefix * "\n"
#   for i = 1:size(data, 1)
#       for j = 1:size(data, 2)
#           this_str = @sprintf("%.6f", data[i, j])
#           str = str * this_str * "f"
#           if (i == size(data, 1) && j == size(data, 2))
#             str = str * ").finished();"
#           else
#             str = str * ","
#           end
#       end
#       str = str * "\n"
#   end
#   str = str * "\n"
#   return str
# end

function export_mat_to_eigen(prefix, data)
  str = prefix * "\n"
  for i = 1:size(data, 1)
      for j = 1:size(data, 2)
          this_str = @sprintf("%.6f", data[i, j])
          str = str * this_str * "f"
          if (i == size(data, 1) && j == size(data, 2))
            str = str * ";"
          else
            str = str * ","
          end
      end
      str = str * "\n"
  end
  str = str * "\n"
  return str
end

@printf("%s",export_mat_to_eigen("A << ", Ã))
@printf("%s",export_mat_to_eigen("B << ", B̃))
@printf("%s",export_mat_to_eigen("Kinf << ", cache.Kinf))
@printf("%s",export_mat_to_eigen("Pinf << ", cache.Pinf))
@printf("%s",export_mat_to_eigen("Quu_inv << ", cache.Quu_inv))
@printf("%s",export_mat_to_eigen("AmBKt << ",   cache.AmBKt))
@printf("%s",export_mat_to_eigen("coeff_d2p << ", cache.coeff_d2p))
@printf("%s",export_mat_to_eigen("Q << ", Q))
@printf("%s",export_mat_to_eigen("R << ", R))
# @printf("%s",export_mat_to_eigen("Kinf_data[NINPUTS*NSTATES]", cache.Kinf2))
# @printf("%s",export_mat_to_eigen("Pinf_data[NSTATES*NSTATES]", cache.Pinf2))
# @printf("%s",export_mat_to_eigen("Quu_inv_data[NINPUTS*NINPUTS]", cache.Quu_inv2))
# @printf("%s",export_mat_to_eigen("AmBKt_data[NSTATES*NSTATES]", cache.AmBKt2))
# @printf("%s",export_mat_to_eigen("coeff_d2p_data[NSTATES*NINPUTS]", cache.coeff_d2p2))

UndefVarError: UndefVarError: Ã not defined

In [168]:
# X_ref_data = [Δx̃ref[i][1:12] for i = 1:Nt]
# # Export a vector of Eigen matrix to C header and source 
# function export_vec_to_c(filename, var_type, var_name, data)
#   declare = "static Eigen::" * var_type * " " * var_name
#   def = declare * " = {\n"
#   for i=1:length(data)
#       def = def * "(Eigen::" * var_type * "() << "
#       for j=1:length(data[1])
#           def = def * string(data[i][j])
#           if j < length(data[1]) 
#               def = def * ","
#           end
#       end
#     def = def * ").finished(),\n"
#   end
#   def = def*"}"

#   open(filename*".h","a") do io
#           println(io, def * ";\n")
#   end
#   return true
# end
# export_vec_to_c("data", "VectorNf", "XrefAll[]", X_ref_data)